#### This notebook explains how to:
(1) Read benchmarking data from S3 via Neuroglancer 

(2) download raw benchmarking data to your local computer 

#### Quick notes on the benchmarking data:

In octree format, data is labled in folders, labeled test_1 through test_25 and validation_1 through validation_25. 

If when downloading, you get a reshape error, try first uploading segments and then re-uploading the volumes. 

Known issues with a few of the files: 

- test_9,test_10 - didnt seem to have good swc alignment

- test_24 - issues with the image

- validation_11 - seems to be a shift between swcs and the image


# Read benchmarking data from S3 with Neuroglancer


##### Define locations

In [7]:
from brainlit.utils import session
from brainlit.utils.swc import graph_to_paths

#Can change to test_"1-25", validation_"1-25"
dest = "s3://open-neurodata/brainlit/benchmarking_data/validation_7" 
dest_segments = "s3://open-neurodata/brainlit/benchmarking_data/validation_7"

##### Create Neuroglancer session & download benchmarking volume

In [8]:
%%capture
sess = session.NeuroglancerSession(url=dest, url_segments=dest_segments, mip=0)  # create session object 
img, bounds, vertices = sess.pull_vertex_list(1, [1], 0, expand=True)  # get full benchmarking image

##### Download a specific .swc

In [9]:
seg_id = 1 # Can change

G = sess.get_segments(seg_id, bounds, rounding = False)
paths = graph_to_paths(G)

Downloading: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 29.49it/s]


##### Download ALL associated .swc's 

In [10]:
#paths = []

#for id in range(1,30):
#    try:
#        G = sess.get_segments(id, bounds, rounding = False)
#        p = graph_to_paths(G)
#        paths.extend(p)
#    except:
#        print('End of seg_id list')
#        break

##### Visualize with napari

In [11]:
import napari
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=1.0, edge_color='blue', opacity=0.8)

# Download RAW benchmarking data with AWS CLI

###### This will download the benchmarking data in .tif and .swc format to a local destination

### 1) Installing AWS CLI

You can install AWS CLI for any major operating system:

#### macOS 
(the full documentation uses pip, but Homebrew works more seamlessly):
Documentation: https://docs.aws.amazon.com/cli/latest/userguide/cli-install-macos.html 

```brew install awscli```
#### Linux 
Documentation: https://docs.aws.amazon.com/cli/latest/userguide/awscli-install-linux.html 

```$ pip install awscli```

#### Windows
Documentation: https://docs.aws.amazon.com/cli/latest/userguide/awscli-install-windows.html
Installer: http://docs.aws.amazon.com/cli/latest/userguide/awscli-install-windows.html

### 2) Configuring AWS CLI

Run ```aws configure``` in any command line and follow the prompts.

You will be asked for 
- AWS Access Key ID
- AWS Secrety Access Key
- Default Region Name: us-east-1, us-east-2, us-west-1, us-west-2, ...
- Default output format: json, table, text

### 3.1) Downloading .tif files

Run the command ```aws s3 sync s3://open-neurodata/brainlit/benchmarking_data/tif-files/ sample-tif-location```

Replace "sample-tif-location" with your respective download destination.

### 3.2) Accessing .tif files

In [ ]:
from pathlib import Path
import numpy as np
from skimage import io

This loops through and reads the .tif files in the download directory. Alter data_dir to point to your download location.

In [ ]:
%%capture
data_dir = Path().resolve().parents[5] / "Downloads" / "sample-tif-location"
im_files = list(data_dir.glob("**/*.tif"))

for im_num, im_file in enumerate(im_files):
    print(f"Image {im_num}/{len(im_files)}")
    print(im_file)
    f = im_file.parts[-1][:-8] + "-gfp.tif" 
    im = io.imread(im_file, plugin="tifffile") 

The below code can visualize a specified .tif file. To visualize tif files with .swc overlaid, run the data through the image mask generation notebook.

In [ ]:
import napari

file_name = "test_10-gfp.tif" # Can change to any image (test 1-25, validation 1-25)

im_file = data_dir / file_name
im = io.imread(im_file, plugin="tifffile") 
    
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(im)

### 4.1) Downloading .swc files

Run the command ```aws s3 sync s3://open-neurodata/brainlit/benchmarking_data/Manual-GT/ sample-tif-location/sample-swc-location```

Replace "sample-swc-location" with your respective download destination. Download it inside the .tif folder to maintain octree format

### 4.2) Accessing .swc files

This loops through the .swc files in the download directory. Alter data_dir to point to your file download location.


In [ ]:
%%capture
data_dir = Path().resolve().parents[5] / "Downloads" / "sample-swc-location" 
swc_files = list(data_dir.glob("**/*.swc"))

for swc_num, swc_file in enumerate(swc_files):
    print(f"SWC {swc_num}/{len(swc_files)}")
    print(swc_file)
    image_name = swc_file.parts[6]
    swc_num = swc_file.parts[7]

### 5) Aligning and visualizing images & swcs

In [ ]:
from brainlit.utils.benchmarking_params import brain_offsets, vol_offsets, scales, type_to_date
from brainlit.utils.swc import read_swc, df_to_graph, graph_to_paths
from pathlib import Path
import numpy as np
from skimage import io
import napari

In [ ]:
im_dir = Path("C:/Users/shrey/Downloads/sample-tif-location")

gfp_files = list(im_dir.glob("**/*-gfp.tif"))

swc_base_path = im_dir / "sample-swc-location"

In [ ]:
%%capture
for im_num, im_path in enumerate(gfp_files):
    
    print(f"Image {im_num+1}/{len(gfp_files)}")
    print(im_path)
    
    f = im_path.parts[-1][:-8].split("_")
    image = f[0]
    date = type_to_date[image]
    num = int(f[1])

    scale = scales[date]
    brain_offset = brain_offsets[date]
    vol_offset = vol_offsets[date][num]
    im_offset = np.add(brain_offset, vol_offset)

    lower = int(np.floor((num - 1) / 5) * 5 + 1)
    upper = int(np.floor((num - 1) / 5) * 5 + 5)
    dir1 = date + "_" + image + "_" + str(lower) + "-" + str(upper)
    dir2 = date + "_" + image + "_" + str(num)
    swc_path = swc_base_path / dir1 / dir2

    swc_files = list(swc_path.glob("**/*.swc"))
    im = io.imread(im_path, plugin="tifffile")
    print(f"Image shape: {im.shape}")

    paths_total = []
    for swc_num, swc in enumerate(swc_files):
        if "0" in swc.parts[-1]:
            # skip the bounding box swc
            continue

        df, swc_offset, _, _, _ = read_swc(swc)

        offset_diff = np.subtract(swc_offset, im_offset)
        G = df_to_graph(df)

        paths = graph_to_paths(G)
       
        for path_num, p in enumerate(paths):
            pvox = (p + offset_diff) / (scale) * 1000
            paths_total.append(pvox)
            
    with napari.gui_qt():
        viewer = napari.Viewer(ndisplay=3)
        viewer.add_image(np.swapaxes(im,0,2))
        viewer.add_shapes(data=paths_total, shape_type='path', edge_width=1.0, edge_color='blue', opacity=0.8)